In [1]:
import numpy as np
from torchvision.datasets import MNIST

### Downloading the MNIST dataset

In [2]:
def download_mnist(is_train: bool):
    dataset = MNIST(root='./data',
                    transform=lambda x: np.array(x).flatten() / 255.0,
                    download=True,
                    train=is_train)
    mnist_data = []
    mnist_labels = []
    for image, label in dataset:
        mnist_data.append(image)
        ohl = np.zeros(10)
        ohl[label] = 1
        mnist_labels.append(ohl)
    return np.array(mnist_data), np.array(mnist_labels)

train_X, train_Y = download_mnist(True)
test_X, test_Y = download_mnist(False)

### Softmax function

In [3]:
def softmax(weighted_sum):
    denominator = np.exp(weighted_sum - np.max(weighted_sum, axis=1, keepdims=True))
    return denominator / np.sum(denominator, axis=1, keepdims=True)

### Predict function

In [4]:
def predict(inputs, weights, bias):
    dot_product = np.dot(inputs, weights) + bias
    return softmax(dot_product)

### Training function

In [5]:
def train(inputs, targets):
    epochs=50
    batch_size=100
    learning_rate=0.0008
    weights = np.zeros((784, 10))
    # weights = np.random.randn(784, 10) * 0.01
    bias = np.zeros(10)
    
    for epoch in range(epochs):
        for start in range(0, inputs.shape[0], batch_size):
            end = start + batch_size
            
            batch_inputs = inputs[start:end]
            batch_targets = targets[start:end]
            
            predictions = predict(batch_inputs, weights, bias)
            
            errors = batch_targets - predictions
            weights += learning_rate * np.dot(batch_inputs.T, errors)
            bias += learning_rate * np.sum(errors, axis=0)
    return weights, bias

weights, bias = train(train_X, train_Y)

### Evaluation

In [6]:
def evaluate_accuracy(inputs, targets, weights, bias):
    predictions = predict(inputs, weights, bias)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(targets, axis=1)
    
    return np.mean(predicted_labels == true_labels)

### Evaluate the model

In [7]:
accuracy = evaluate_accuracy(test_X, test_Y, weights, bias)
print(f"Accuracy: {(accuracy * 100):.2f}%")

Accuracy: 92.50%
